# User Guide

In [18]:
import ipcoal
import toytree
import numpy as np

## Workflow

The user interacts with ipcoal through the Model object, through which the following are defined:
* Tree topology
* Effective population size
* Admixture
* Substitution rate
* Recombination rate

Then, the methods `.sim_loci()` or `.sim_snps()` generate data based on the input parameters.

The resulting simulated data consists of a dataframe object that holds the simulated genealogies, and matrices that contain the actual simulated sequences.

After data simulation with `.sim_loci`, ipcoal can automatically infer a gene tree from the simulated data at each locus. These gene trees are then incorporated into the dataframe object.

## Defining the tree topology.

`ipcoal` uses a `toytree` tree object as input for the toplogy of the species tree. You can read more about toytree at [the toytree docs](https://toytree.readthedocs.io/en/latest/index.html).

A `toytree` tree object can be generated three different ways:

#### random topology

In [6]:
# define the tree topology
tre = toytree.rtree.coaltree(5)
tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tad6283743fda4095a9055828fe85d0e5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 0.0065
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 0.0065
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 0.0939
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 0.0584
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 0.0584
support: 100
height: 0.0000 4 idx: 5
name: 5
dist: 0.0874
support: 100
height: 0.0065 5 idx: 6
name: 6
dist: 0.1216
support: 100
height: 0.0939 6 idx: 7
name: 7
dist: 0.1572
support: 100
height: 0.0584 7 idx: 8
name: 8
dist: 0.0001
support: 100
height: 0.2155 8 0.0 0.1 0.2

Using the coaltree function multiple times can give different topologies and different node heights:

In [7]:
# generate 3 randome trees, save as a multitree object
mtre = toytree.mtree([toytree.rtree.coaltree(5) for i in range(3)])
# plot the trees
mtre.draw_tree_grid(
    start=0, 
    nrows=1, ncols=3, 
    shared_axis=True,
    edge_type='c',
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t91e7f2f16e064ce9bfc1b97af8bb9706" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 525.0 250.0" width="525.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r0 r1 r2 r3 r4 r0 r1 r2 r3 r4 0.0 0.3 0.6

Node heights can be randomly shifted using the `node_slider()` method:

In [8]:
# apply `node_slider()` three times to the first tree in our multitree object
nodeslide_trees = toytree.mtree([mtre.treelist[0].mod.node_slider() for i in range(3)])
# plot the nodeslider trees:
nodeslide_trees.draw_tree_grid(
    start=0, 
    nrows=1, ncols=3, 
    shared_axis=True,
    edge_type='c',
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t6e0d33fdaaa94cc7abf5222fa5458cf1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 525.0 250.0" width="525.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r0 r1 r2 r3 r4 r0 r1 r2 r3 r4 0.0 0.2 0.4

#### balanced/imbalanced topology

In [9]:
# define a 6-tip balanced tree
tre = toytree.rtree.baltree(6)
tre.draw(tree_style='c');
# define a 6-tip imbalanced tree
tre = toytree.rtree.imbtree(6)
tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tbd639efd4736456198aed51a8efd3a49" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r1
dist: 0.3333
support: 100
height: 0.0000 0 idx: 1
name: r2
dist: 0.3333
support: 100
height: 0.0000 1 idx: 2
name: r0
dist: 0.6667
support: 100
height: 0.0000 2 idx: 3
name: r4
dist: 0.3333
support: 100
height: 0.0000 3 idx: 4
name: r5
dist: 0.3333
support: 100
height: 0.0000 4 idx: 5
name: r3
dist: 0.6667
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 0.3333
support: 100
height: 0.3333 6 idx: 7
name: 7
dist: 0.3333
support: 100
height: 0.3333 7 idx: 8
name: 8
dist: 0.3333
support: 100
height: 0.6667 8 idx: 9
name: 9
dist: 0.3333
support: 100
height: 0.6667 9 idx: 10
name: 10
dist: 0.3333
support: 100
height: 1.0000 10 0.0 0.5 1.0

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t08fdc7951a644a899edd876eddc3b1ed" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r1
dist: 0.2000
support: 100
height: 0.0000 0 idx: 1
name: r0
dist: 0.2000
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 0.4000
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 0.6000
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 0.8000
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 1.0000
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 0.2000
support: 100
height: 0.2000 6 idx: 7
name: 7
dist: 0.2000
support: 100
height: 0.4000 7 idx: 8
name: 8
dist: 0.2000
support: 100
height: 0.6000 8 idx: 9
name: 9
dist: 0.2000
support: 100
height: 0.8000 9 idx: 10
name: 10
dist: 0.2000
support: 100
height: 1.0000 10 0.0 0.5 1.0

#### topology from newick

In [10]:
tre = toytree.tree(
             '((r5:2.62256,(r4:0.96563,r3:0.96563)100:1.65693)100:\
              9.63792,(r2:7.42816,(r1:4.0738,r0:4.0738)100:\
              3.35436)100:4.83232);'
            )
tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t3e28207902df4131ba87017ca90ab64e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 4.0738
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 4.0738
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 7.4282
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 0.9656
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 0.9656
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 2.6226
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 3.3544
support: 100
height: 4.0738 6 idx: 7
name: 7
dist: 1.6569
support: 100
height: 0.9656 7 idx: 8
name: 8
dist: 4.8323
support: 100
height: 7.4282 8 idx: 9
name: 9
dist: 9.6379
support: 100
height: 2.6226 9 idx: 10
name: 10
dist: 1.0000
support: 100
height: 12.2605 10 0 6 12

#### Rescaling tree height

`ipcoal` interprets node heights as units of generations, so on phylogenetic scales we expect the node heights to be of pretty large magnitudes.

In [11]:
tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t8292437b9500449ab5370dc93e3c582a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 4.0738
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 4.0738
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 7.4282
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 0.9656
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 0.9656
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 2.6226
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 3.3544
support: 100
height: 4.0738 6 idx: 7
name: 7
dist: 1.6569
support: 100
height: 0.9656 7 idx: 8
name: 8
dist: 4.8323
support: 100
height: 7.4282 8 idx: 9
name: 9
dist: 9.6379
support: 100
height: 2.6226 9 idx: 10
name: 10
dist: 1.0000
support: 100
height: 12.2605 10 0 6 12

In [12]:
rescale_tre = tre.mod.node_scale_root_height(treeheight=1e6)
rescale_tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t2d8e562bee76456e88dc06e009702e03" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 332270.8409
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 332270.8409
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 605862.0870
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 78759.5592
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 78759.5592
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 213903.5340
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 273591.2460
support: 100
height: 332270.8409 6 idx: 7
name: 7
dist: 135143.9748
support: 100
height: 78759.5592 7 idx: 8
name: 8
dist: 394137.9130
support: 100
height: 605862.0870 8 idx: 9
name: 9
dist: 786096.4660
support: 100
height: 213903.5340 9 idx: 10
name: 10
dist: 81562.8752
support: 100
height: 1000000.0000 10 0 500000 1000000

## Defining the demographic model.

The `ipcoal` Model object allows users to designate effective population size, admixture, substitution rates, and recombination rates.

### Effective population size (Ne)

First, need to define a tree drawing style for the next few examples.

#### Define our tree (from previous section)

In [20]:
tre = toytree.tree(
             '((r5:2.62256,(r4:0.96563,r3:0.96563)100:1.65693)100:\
              9.63792,(r2:7.42816,(r1:4.0738,r0:4.0738)100:\
              3.35436)100:4.83232);'
            ).mod.node_scale_root_height(treeheight=1e6)
tre.draw(tree_style='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t555ec0f8569548a5b5f200c64bb3b716" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r0
dist: 332270.8409
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 332270.8409
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 605862.0870
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 78759.5592
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 78759.5592
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 213903.5340
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 273591.2460
support: 100
height: 332270.8409 6 idx: 7
name: 7
dist: 135143.9748
support: 100
height: 78759.5592 7 idx: 8
name: 8
dist: 394137.9130
support: 100
height: 605862.0870 8 idx: 9
name: 9
dist: 786096.4660
support: 100
height: 213903.5340 9 idx: 10
name: 10
dist: 81562.8752
support: 100
height: 1000000.0000 10 0 500000 1000000

#### Set one Ne for the whole tree

In [22]:
# define a model with Ne = 2
mod = ipcoal.Model(tree=tre, 
                      Ne=2, 
                      seed=123)

# or define a model with Ne = 1e5
mod = ipcoal.Model(tree=tre, 
                      Ne=1e5, 
                      seed=123)

In [ ]:
model3 = ipcoal.Model(tree=tre, seed=12345)

#### Set branch-specific Ne

First, set an "Ne" attribute for the tree object.

In [23]:
# set specific values to some nodes and a global default
tre = tre.set_node_values(
    attr="Ne", 
    # the branches above these nodes get specific values
    values={8: 5e6, 6: 5e6, 2: 5e6},
    # the rest of the branches take this default value
    default=20,
)

Look at the tree:

In [24]:
# draw to show edge widths and node idxs
tre.draw(
    width=300, height=250,
    edge_widths=np.log(tre.get_edge_values("Ne")),
    tree_style='c',
    tip_labels=True,
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t41345df00b5d4bd1b576d75e63fb7ec7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 300.0 250.0" width="300.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 idx: 0
name: r0
dist: 332270.8409
support: 100
height: 0.0000 0 idx: 1
name: r1
dist: 332270.8409
support: 100
height: 0.0000 1 idx: 2
name: r2
dist: 605862.0870
support: 100
height: 0.0000 2 idx: 3
name: r3
dist: 78759.5592
support: 100
height: 0.0000 3 idx: 4
name: r4
dist: 78759.5592
support: 100
height: 0.0000 4 idx: 5
name: r5
dist: 213903.5340
support: 100
height: 0.0000 5 idx: 6
name: 6
dist: 273591.2460
support: 100
height: 332270.8409 6 idx: 7
name: 7
dist: 135143.9748
support: 100
height: 78759.5592 7 idx: 8
name: 8
dist: 394137.9130
support: 100
height: 605862.0870 8 idx: 9
name: 9
dist: 786096.4660
support: 100
height: 213903.5340 9 idx: 10
name: 10
dist: 81562.8752
support: 100
height: 1000000.0000 10 0 500000 1000000

Now define our model. The "Ne" attribute of the tree will, by default, be the `Ne` value used by the model.

In [26]:
mod = ipcoal.Model(tree=tre, 
                      seed=123)

### Admixture

In [27]:
# init a random tree
tre = toytree.rtree.unittree(8, treeheight=1e6, seed=123)

In [28]:
# draw with one admixture edge
tre.draw(
    ts='c',
    admixture_edges=(4, 3, 0.5, 0.1),
);

# draw with two admixture edges
tre.draw(
    ts='c',
    admixture_edges=[
        (4, 3, 0.5, 0.1),
        (3, 8, 0.5, 0.2),
    ],
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t37a611f6f528423fa673f9f27b568c6e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r6
dist: 500000
support: 100
height: 0 0 idx: 1
name: r4
dist: 500000
support: 100
height: 0 1 idx: 2
name: r7
dist: 500000
support: 100
height: 0 2 idx: 3
name: r2
dist: 500000
support: 100
height: 0 3 idx: 4
name: r5
dist: 250000
support: 100
height: 0 4 idx: 5
name: r0
dist: 250000
support: 100
height: 0 5 idx: 6
name: r3
dist: 500000
support: 100
height: 0 6 idx: 7
name: r1
dist: 750000
support: 100
height: 0 7 idx: 8
name: 8
dist: 250000
support: 100
height: 250000 8 idx: 9
name: 9
dist: 250000
support: 100
height: 500000 9 idx: 10
name: 10
dist: 250000
support: 100
height: 500000 10 idx: 11
name: 11
dist: 250000
support: 100
height: 500000 11 idx: 12
name: 12
dist: 250000
support: 100
height: 750000 12 idx: 13
name: 13
dist: 250000
support: 100
height: 750000 13 idx: 14
name: 14
dist: 250000
support: 100
height: 1000000 14 0 500000 1000000

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tdba9d0a4dcdf4b5dac6c409dee23536f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r6
dist: 500000
support: 100
height: 0 0 idx: 1
name: r4
dist: 500000
support: 100
height: 0 1 idx: 2
name: r7
dist: 500000
support: 100
height: 0 2 idx: 3
name: r2
dist: 500000
support: 100
height: 0 3 idx: 4
name: r5
dist: 250000
support: 100
height: 0 4 idx: 5
name: r0
dist: 250000
support: 100
height: 0 5 idx: 6
name: r3
dist: 500000
support: 100
height: 0 6 idx: 7
name: r1
dist: 750000
support: 100
height: 0 7 idx: 8
name: 8
dist: 250000
support: 100
height: 250000 8 idx: 9
name: 9
dist: 250000
support: 100
height: 500000 9 idx: 10
name: 10
dist: 250000
support: 100
height: 500000 10 idx: 11
name: 11
dist: 250000
support: 100
height: 500000 11 idx: 12
name: 12
dist: 250000
support: 100
height: 750000 12 idx: 13
name: 13
dist: 250000
support: 100
height: 750000 13 idx: 14
name: 14
dist: 250000
support: 100
height: 1000000 14 0 500000 1000000

We define these admixture edges the same way as input for the the model object.

In [ ]:
# init a model Class object for simulations
model = ipcoal.Model(
    tree=tre,
    admixture_edges=[
        (3, 4, 0.5, 0.2),
        (9, 10, 0.5, 0.2),
    ],
    seed=123,
)

By default, the admixture event occurs as a single pulse at a moment in time (`admixture_type=0`).

Admixture can also be simulated at a constant rate over a specified time interval:

In [ ]:
# init a model Class object for simulations
model = ipcoal.Model(
    tree=tre,
    # "1" type specifies interval migration
    admixture_type=1,
    admixture_edges=[
        (3, 4, (0.2,0.5), 0.2),
        (9, 10, (0.4,0.7), 0.2),
    ],
    seed=123,
)

### Subsitution and recombination

In [ ]:
# init a random tree
tre = toytree.rtree.unittree(8, treeheight=1e6, seed=123)

A single mutation rate argument can be specified, which determines the scaling of gene tree heights when passed to seq-gen for producing sequences.

In [ ]:
# init a model Class object for simulations
model = ipcoal.Model(
    tree=tre,
    mut=1e-8,
    seed=123,
)

A single recombination rate argument can be specified, which is passed to msprime for producing genealogies.

In [ ]:
# init a model Class object for simulations
model = ipcoal.Model(
    tree=tre,
    recomb=1e-8,
    seed=123,
)

## Simulating unlinked SNPs

In [ ]:
# simulate N unlinked SNPs (will run until N snps are produced)
model.sim_snps(1000)

In [ ]:
model.df.head(10)

## Simulating loci

In [ ]:
# simulate N loci of len L 
model.sim_loci(100, 3000)

In [ ]:
# view the genealogies and stats in a table
model.df

In [ ]:
# view sequence data as an array
model.seqs

## Inferring gene trees

* For use with multilocus data.

Once multilocus data has been simulated, the `.infer_gene_trees()` method automates the gene tree inference process across all loci.

In [ ]:
# define the tree topology
tre = toytree.rtree.unittree(5, treeheight=1e6, seed=111)
# define 
model = ipcoal.Model(tree=tre, Ne=1e6, recomb=1e-9, seed=111)
# infer a tree for every locus
model.infer_gene_trees(inference_method='raxml')

Alternative inference methods include `'mrbayes'` and `'iqtree'`.

After running gene tree inference, our inferred gene trees can be found in the new `inferred_tree` column of our dataframe:

In [ ]:
model.df.head(10)

## Writing to disk

In [ ]:
# save table to a CSV file
model.df.to_csv("./tree_table.csv")

# write loci as separate phylip files to a directory
model.write_loci_to_phylip(outdir="./tests")

# write concatenated loci or snps to a phylip file
model.write_concat_to_phylip(outdir="./tests", name="test.phy")